<a href="https://colab.research.google.com/github/dani1925/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-21 02:37:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.92MB/s    in 0.2s    

2022-01-21 02:37:31 (4.92 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

### Create DataFrames to match tables

In [16]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [35]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15634680|             1|
|   44696507|             1|
|   19901367|             1|
|   11337682|             1|
|   52021773|             1|
|   29916198|             1|
|   42030944|             1|
|   52765209|             1|
|   13519509|             2|
|   39320350|             5|
+-----------+--------------+
only showing top 10 rows



In [18]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)



+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00TQ71BXI|Moonlight Mysteri...|
|B00CHZG1FE|Image Line FL Stu...|
|B002JB3BC2|Anti-Virus & Anti...|
|B0113MZ1AC|Windows 8 64 Bit ...|
|B00003069T|ClueFinders 5th G...|
|B008EQUD4U|   Camtasia Studio 8|
|B008IVNFDM|Learn Adobe InDes...|
|B001Q24UAM|Play! The Ultimat...|
|B005HMUUSI|Mariner StoryMill...|
|B00005MG8G|My Deluxe MailLis...|
+----------+--------------------+
only showing top 10 rows



In [29]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|   42605767|B00MUTIDKI|     248732228| 2015-08-31|
|R12NR0R5A9F7FT|   51771800|B00EPACNUG|     531462352| 2015-08-31|
|R1LSH74R9XAP59|   16053526|B00164AZA4|     473982505| 2015-08-31|
|R1QXUNTF76K7L6|   15319481|B00E6LIEFM|     189774198| 2015-08-31|
|R2F7DR75PS8NKT|    1441820|B00VWEBG06|     852470365| 2015-08-31|
|R2C1DJSCC8UFS6|   37107850|B00EP7AP7C|     279360628| 2015-08-31|
|R1AXGS1W4YFXMX|     302120|B00OW2PET4|     729971168| 2015-08-31|
|R1XU1B93402SYJ|   20193077|B00N4OLCRO|     776572654| 2015-08-31|
|R2U432NB3OPVR0|   13106017|B005CELN8W|     222071424| 2015-08-31|
|R3R6FIMIOQ5SP9|   32587108|B005CELL1G|     168801430| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [30]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [31]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database.cusa4qve384u.us-east-1.rds.amazonaws.com:5432/challenge"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [32]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [33]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [36]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [37]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# DELIVERABLE 2


In [49]:
import pandas as pd
new_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
new_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

In [52]:
pd_df = new_df.toPandas()
pd_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R3EFW2STIYIY0I,1,2,2,N,Y
1,R12NR0R5A9F7FT,5,0,0,N,Y
2,R1LSH74R9XAP59,2,0,1,N,Y
3,R1QXUNTF76K7L6,2,0,0,N,Y
4,R2F7DR75PS8NKT,5,0,0,N,Y


In [54]:
total_votes = pd_df.loc[pd_df['total_votes']>=20]
total_votes.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
19,R26CC1STPSUHBS,1,27,39,N,N
202,R7K83D59T9KWS,1,16,26,N,N
219,R2ATG5BPVILHUM,1,16,25,N,N
345,R1Q71TAM5ZOYTK,3,57,61,N,N
454,RR8HYI7C2XKMU,5,25,25,N,Y


Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.


In [57]:
helpf_total = total_votes.loc[total_votes['helpful_votes']  / total_votes['total_votes']>=0.5]
helpf_total.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
19,R26CC1STPSUHBS,1,27,39,N,N
202,R7K83D59T9KWS,1,16,26,N,N
219,R2ATG5BPVILHUM,1,16,25,N,N
345,R1Q71TAM5ZOYTK,3,57,61,N,N
454,RR8HYI7C2XKMU,5,25,25,N,Y


In [60]:
vine_Y = helpf_total.loc[helpf_total['vine']=='Y']
vine_Y.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
11313,R28EIR3Z6RRUNN,2,50,60,Y,N
19233,R2YO2RESXZEWOF,4,24,25,Y,N
30450,R8GEE2EW3D6S9,3,49,54,Y,N
32929,R33W9V1MX6WKPS,4,47,53,Y,N
35028,R24E3W2ZRIFG8C,5,91,93,Y,N


In [61]:
vine_N = helpf_total.loc[helpf_total['vine']=='N']
vine_N.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
19,R26CC1STPSUHBS,1,27,39,N,N
202,R7K83D59T9KWS,1,16,26,N,N
219,R2ATG5BPVILHUM,1,16,25,N,N
345,R1Q71TAM5ZOYTK,3,57,61,N,N
454,RR8HYI7C2XKMU,5,25,25,N,Y


In [75]:
total_reviews = vine_Y['review_id'].count()
print(total_reviews)

total_reviews_N = vine_N['review_id'].count()
print(total_reviews_N)

248
17514


In [76]:
star_vine_y = vine_Y['star_rating'].loc[vine_Y['star_rating']==5].count()
print(star_vine_y)


star_vine_n = vine_N['star_rating'].loc[vine_N['star_rating']==5].count()
print(star_vine_n)

102
5154


In [78]:
five_stars_Y = star_vine_y / total_reviews
print(five_stars_Y*100)

five_stars_n = star_vine_n / total_reviews_N
print(five_stars_n*100)

41.12903225806452
29.427886262418635
